In [13]:
#task 5 - Choi Matrix of probabilistic ensemble

from qiskit import *
from qiskit.quantum_info import Choi
import numpy as np


#code previously developed for task 4 that creates and load the txt file with the probabilistic ensemble:

def create_prob_cir_txt_file(circuit_list,prob_distr):
    if len(circuit_list) == len(prob_distr):
        with open("probs_circuits1.txt", "w") as txt_file:
            for i in prob_distr:
                txt_file.write(str(i)+'\n')
            for i in circuit_list:
                txt_file.write(i.qasm())
    else:
        print("Input error = the number of circuits must be equal to the number of probabilities")
        
def  load_prob_cir_txt_file(txt_file):
    result =[]
    circuits =[]
    probs =[]
    with open(txt_file) as f:
        lines = f.readlines()
        #Reading and loading probabilities
        j = 0
        while(lines[j] != 'OPENQASM 2.0;\n'):
            probs.append(float(lines[j]))
            j+=1
        #Reading and loading circuits
        for i in range(len(probs)):
            qasm_str = lines[j]  #qasm header  
            j+=1
            while(lines[j] != 'OPENQASM 2.0;\n'):
                qasm_str = qasm_str+lines[j]
                j+=1
                if j >= len(lines): break
            circ = QuantumCircuit.from_qasm_str(qasm_str)
            circuits.append(circ) 
        #Constructing output result
        if len(probs) == len(circuits):
            for i in range(len(probs)):
                result.append([probs[i],circuits[i]])
        else:
            print("Input error = the number of circuits must be equal to the number of probabilities")
    return result
        

#task 5
#new code creating the choi matrix of the probabilistic ensemble.
    
def create_choi_matrix(n, txt_file):
    probs_circuits = load_prob_cir_txt_file(txt_file)
    probs =[i[0] for i in probs_circuits]
    circuits = [i[1] for i in probs_circuits]
    choi_ensemble = 0
    if int(sum(probs)) < 1:
        circ_identity = QuantumCircuit(n,n,name='Id')
        circ_identity.i(range(n)) #does nothing
        circuits.append(circ_identity)
        probs.append(round(1 - sum(probs),3))
    for j in range(len(circuits)):
            choi_circuit = quantum_info.Choi(circuits[j]).data / (2**n)
            choi_ensemble += probs[j]*choi_circuit
    return choi_ensemble
    
        
    

In [14]:
#Creating txt file with probs and circuits
n = 1
c1 = QuantumCircuit(n,n,name='X')
c1.x(range(n))
c2 = QuantumCircuit(n,n,name='Y')
c2.y(range(n))
c3 = QuantumCircuit(n,n,name='Z')
c3.z(range(n))

create_prob_cir_txt_file([c1,c2,c3],[0.25,0.25,0.25])  

In [15]:
#Reading and loading txt file with probs and circuits for verification
txt_file = 'probs_circuits1.txt'
probs_circuits = load_prob_cir_txt_file(txt_file)
probs_circuits  

[[0.25, <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x168efbceac8>],
 [0.25, <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x168efb92fc8>],
 [0.25, <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x168efbf7c08>]]

In [16]:
#Creating choi matrix for the probabilistic ensemble
n = 1  
path = 'probs_circuits1.txt'
choi = create_choi_matrix(n,path)
choi

array([[0.25+0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.25+0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.25+0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, 0.25+0.j]])